In [13]:
import cv2;
import math as m;
import numpy as np; # без векторизации тут слишком медленно, грузит одно ядро на 12%,
                    # а обработка кадра занимает секунд по 15-20

In [56]:
def applyFilter(source, coefficient = 2):
    vid = cv2.VideoCapture(source);
    
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH));
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT));
    fps = vid.get(cv2.CAP_PROP_FPS);
    
    dest = cv2.VideoWriter('out_' + source, int(vid.get(cv2.CAP_PROP_FOURCC)), fps, (width, height));
    
    counter = 1;
    while vid.isOpened():
        ret, frame = vid.read();
        if not ret:break;
        
        frame = frame.astype(np.float32);
        frame = (np.sin(coefficient * counter / fps) + 1) * (frame - 128) + 128;
        frame = np.clip(frame, 0, 255);
        frame = frame.astype(np.uint8);

        dest.write(frame);
        counter += 1;

    vid.release();
    dest.release();
    print("Обработка завершена!");
    
def applyConcat(src1, src2, concatTime = 1):
    vid1 = cv2.VideoCapture(src1);
    vid2 = cv2.VideoCapture(src2);

    width = int(vid1.get(cv2.CAP_PROP_FRAME_WIDTH));
    height = int(vid1.get(cv2.CAP_PROP_FRAME_HEIGHT));
    fps = vid1.get(cv2.CAP_PROP_FPS);
    framesVid1 = int(vid1.get(cv2.CAP_PROP_FRAME_COUNT));

    dest = cv2.VideoWriter('concatted_' + src1, int(vid1.get(cv2.CAP_PROP_FOURCC)), fps, (width, height));

    concatFrames = int(fps * concatTime);

    frameCount = 0
    while vid1.isOpened() and vid2.isOpened():
        ret1, frame1 = vid1.read();
        ret2, frame2 = vid2.read();

        
        if frameCount >= framesVid1:
            if not ret2: break;
            dest.write(frame2);
        else:
            if not ret1 or not ret2: break;
            if frameCount > framesVid1 - concatFrames:
                progress = (frameCount - (framesVid1 - concatFrames)) / concatFrames;
                sectionsToReplace = int(progress * width);
                frame1[:, -sectionsToReplace:] = frame2[:, :sectionsToReplace];
            dest.write(frame1);

        
        frameCount += 1;

    vid1.release();
    vid2.release();
    dest.release();
    print("Склейка завершена!");
    

In [57]:
applyFilter('example.mp4');

Обработка завершена!


In [58]:
applyConcat('example.mp4', 'example2.mp4');

Склейка завершена!
